In [42]:
### Import necessary packages
import pandas as pd
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import statsmodels.formula.api as smf
    # from textbook but requires combined X and y

In [ ]:
### ORIGINAL MERGED CSV
# read original csv file into DataFrame
# data = pd.read_csv("airline.csv")

# print(data.shape)
    # (7065617, 120)

In [ ]:
### cLEANED BUT UNFILTERED CSV
# read csv file into DataFrame (cleaned data)
# clean = pd.read_csv("airline_data_clean.csv")

# print(clean.shape)
    # (7065617, 69)


### Initial DataFrame
read cleaned and filtered csv file (top_ports_lines.csv) in as a DataFrame

In [43]:
# READ CLEANED AND FILTERED CSV FILE INTO DATAFRAME
df = pd.read_csv("top_ports_lines.csv")
# print(df.head())
# print(df.shape)
    # (1672943, 69)

In [44]:
# retrieve the data types of all columns in the full DataFrame
column_info = df.dtypes

# print(column_info)

A quick look at the available data for our target variable of interest.

In [30]:
print(f"The total number of cancellations is {df["cancelled"].sum()}.")
print(f"The total number of provided reasons for flights being canceled is {df["cancellation_code"].notna().sum()}.")
print(f"The number of cancellations due to carrier is {sum(df["cancellation_code"] == "A")}.")
print(f"The number of cancellations due to weather is {sum(df["cancellation_code"] == "B")}.")
print(f"The number of cancellations due to national air system is {sum(df["cancellation_code"] == "C")}.")
print(f"The number of cancellations due to security is {sum(df["cancellation_code"] == "D")}.")

The total number of cancellations is 1705972.
The total number of provided reasons for flights being canceled is 33029.
The number of cancellations due to carrier is 14199.
The number of cancellations due to weather is 16438.
The number of cancellations due to national air system is 2262.
The number of cancellations due to security is 130.


In [45]:
df['cancelled'] = df['cancelled'].map({1:0, 2:1})

### Train-Validation-Test Split

Split the DataFrame into X (features) and y (target variable)

In [46]:
y = df[["cancelled"]]
X = df.drop("cancelled", axis=1) #dropping the column "cancelled"

Selecting which variables we think are important to use in our logistic regression model, a priori.

In [33]:
X = X[["operating_airline", "origin", "crs_dep_time", "dep_delay",
                   "distance", "carrier_delay", "weather_delay", "nas_delay",
                   "security_delay", "late_aircraft_delay"]]

One-hot encode the categorical variables ("operating_airline" and "origin")

In [47]:
# define one hot encoding
encoder = OneHotEncoder(sparse_output=False, drop="first")
# transform data

encoder.fit(X[["operating_airline", "origin"]])

onehot = encoder.transform(X[["operating_airline", "origin"]])
# print(onehot)

# get column names
col_names = encoder.get_feature_names_out(["operating_airline", "origin"])

# Create DataFrame with proper column names
one_hot_df = pd.DataFrame(onehot, columns=col_names)


In [52]:
one_hot_df.head()

,operating_airline_AS,operating_airline_B6,operating_airline_DL,operating_airline_F9,operating_airline_G4,operating_airline_HA,operating_airline_NK,operating_airline_UA,operating_airline_WN,origin_CLT,origin_DEN,origin_DFW,origin_LAS,origin_LAX,origin_MCO,origin_MIA,origin_ORD,origin_PHX
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
x_encoded = pd.concat([X, one_hot_df], axis=1)

In [54]:
x_encoded.columns

Index(['year', 'quarter', 'month', 'dayof_month', 'day_of_week', 'flight_date',
       'marketing_airline_network', 'operated_or_branded_code_share_partners',
       'dot_id_marketing_airline', 'iata_code_marketing_airline',
       'flight_number_marketing_airline', 'operating_airline',
       'dot_id_operating_airline', 'iata_code_operating_airline',
       'tail_number', 'flight_number_operating_airline', 'origin_airport_id',
       'origin_airport_seq_id', 'origin_city_market_id', 'origin',
       'origin_city_name', 'origin_state', 'origin_state_fips',
       'origin_state_name', 'origin_wac', 'dest_airport_id',
       'dest_airport_seq_id', 'dest_city_market_id', 'dest', 'dest_city_name',
       'dest_state', 'dest_state_fips', 'dest_state_name', 'dest_wac',
       'crs_dep_time', 'dep_time', 'dep_delay', 'dep_delay_minutes',
       'dep_del15', 'departure_delay_groups', 'dep_time_blk', 'taxi_out',
       'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time',
       'ar

In [55]:
model_encoded = pd.concat([x_encoded, y], axis=1)
model_logit = smf.logit('cancelled ~ crs_dep_time + dep_delay + distance + carrier_delay + weather_delay + nas_delay + security_delay + late_aircraft_delay', data = model_encoded).fit()
#'operating_airline_AS + operating_airline_B6 +operating_airline_DL + operating_airline_F9 + operating_airline_G4 + operating_airline_HA + operating_airline_NK + operating_airline_UA + operating_airline_WN + origin_CLT + origin_DEN + origin_DFW + origin_LAS + origin_LAX + origin_MCO + origin_MIA + origin_ORD + origin_PHX', data=model_encoded).fit()
model_logit.summary()

c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)


         Current function value: 0.000000
         Iterations: 35


c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\ProgramData\anaconda3\envs\irvin_env\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSe

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              cancelled   No. Observations:               402519
Model:                          Logit   Df Residuals:                   402510
Method:                           MLE   Df Model:                            8
Date:                Thu, 10 Jul 2025   Pseudo R-squ.:                     inf
Time:                        15:51:02   Log-Likelihood:            -4.2475e-06
converged:                      False   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.6797   7025.374  -9.68e-05      1.000   -1.38e+04    1.38e+04
crs_dep_time           -0.0226      7.409     -0.003      0.998     -14.543      14.498
dep_delay              -0.0821     86.947     -0.001      0.999    -170.495     170.331
distance               -0.0181      9.209     -0.002      0.998     -18.068      18.032
carrier_delay           0.0352    104.903      0.000      1.000    -205.571     205.641
weather_delay           0.0614    148.397      0.000      1.000    -290.792     290.915
nas_delay              -0.0903     72.432     -0.001      0.999    -142.055     141.874
security_delay         -0.0843    578.728     -0.000      1.000   -1134.371    1134.202
late_aircraft_delay     0.0648    117.941      0.001      1.000    -231.095     231.224
=======================================================================================

Complete Separation: The results show that there iscomplete separation or perfect prediction.
In this case the Maximum Likelihood Estimator does not exist and the parameters
are not identified.
"""

In [58]:
model_encoded["carrier_delay"] = model_encoded["carrier_delay"].fillna(0)
model_encoded["weather_delay"] = model_encoded["weather_delay"].fillna(0)
model_encoded["nas_delay"] = model_encoded["nas_delay"].fillna(0)
model_encoded["security_delay"] = model_encoded["security_delay"].fillna(0)
model_encoded["late_aircraft_delay"] = model_encoded["late_aircraft_delay"].fillna(0)
model_encoded["dep_delay"] = model_encoded['dep_delay'].fillna(model_encoded["dep_delay"].median())
model_encoded["crs_dep_hour"] = model_encoded["crs_dep_time"].astype(str).str.zfill(4).str[:2].astype(int)

In [57]:
model_encoded.to_csv("final_dta.csv", index=False)

Create 50:40:10 train:validation:test split before performing exploratory data analysis to prevent data leakage and overfitting. Model development and feature selection will be done on the training set only.

In [ ]:
### Train, Validation, Test Split

x_train, x_valtest, y_train, y_valtest = train_test_split(x_encoded, y, train_size=0.50, random_state=123)
x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, train_size=0.8, random_state=123)

In [ ]:
# print(x_train.shape)
#     # (836471, 68) 
# print(x_val.shape)
#     # (669177, 68)
# print(x_test.shape)
#     # (167295, 68)
# print(y_train.shape)
#     # (836471,)
# print(y_val.shape)
#     # (669177,)
# print(y_test.shape)
#     # (167295,)

### Exploratory Data Analysis

First, let's take a look at our target variable.

In [ ]:
print(y_train.value_counts())
    # currently, the binary variable has values 1 and 2 when originally they were 0 (no) and 1 (yes)

# transform binary variable "cancelled" with values 1 and 2 to 0 and 1, respectively
y_train = y_train - 1
y_val = y_val - 1
# y_test = y_test - 1

print(y_train.value_counts())

In [ ]:
### Original selecting which variables we think are important to use in our logistic regression model, a priori.
# x_train = x_train[["operating_airline", "origin", "crs_dep_time", "dep_delay",
#                    "distance", "carrier_delay", "weather_delay", "nas_delay",
#                    "security_delay", "late_aircraft_delay"]]

# x_val = x_val[["operating_airline", "origin", "crs_dep_time", "dep_delay",
#                    "distance", "carrier_delay", "weather_delay", "nas_delay",
#                    "security_delay", "late_aircraft_delay"]]

# x_test = x_test[["operating_airline", "origin", "crs_dep_time", "dep_delay",
#                    "distance", "carrier_delay", "weather_delay", "nas_delay",
#                    "security_delay", "late_aircraft_delay"]]

# x_train.head(10)

Only keep the hour when the flight was expected to depart

In [ ]:
### create new var crs_dep_hour to only include the expected hour of departure and drop the crs_dep_time var
x_train["crs_dep_hour"] = x_train["crs_dep_time"].astype(str).str.zfill(4).str[:2].astype(int)
x_train = x_train.drop(columns=["crs_dep_time"])

x_val["crs_dep_hour"] = x_val["crs_dep_time"].astype(str).str.zfill(4).str[:2].astype(int)
x_val = x_val.drop(columns=["crs_dep_time"])

In [ ]:
# Assumption: if na, that observation was not affected by that particular delay reason
x_train["carrier_delay"] = x_train["carrier_delay"].fillna(0)
x_train["weather_delay"] = x_train["weather_delay"].fillna(0)
x_train["nas_delay"] = x_train["nas_delay"].fillna(0)
x_train["security_delay"] = x_train["security_delay"].fillna(0)
x_train["late_aircraft_delay"] = x_train["late_aircraft_delay"].fillna(0)

x_val["carrier_delay"] = x_val["carrier_delay"].fillna(0)
x_val["weather_delay"] = x_val["weather_delay"].fillna(0)
x_val["nas_delay"] = x_val["nas_delay"].fillna(0)
x_val["security_delay"] = x_val["security_delay"].fillna(0)
x_val["late_aircraft_delay"] = x_val["late_aircraft_delay"].fillna(0)

In [ ]:
# impute missing values in dep_delay using the median (data is skewed)
x_train["dep_delay"] = x_train['dep_delay'].fillna(x_train["dep_delay"].median())

x_val["dep_delay"] = x_val['dep_delay'].fillna(x_val["dep_delay"].median())

In [ ]:
x_train["carrier_delay"] = x_train["carrier_delay"].astype(int)
x_train["weather_delay"] = x_train["weather_delay"].astype(int)
x_train["nas_delay"] = x_train["nas_delay"].astype(int)
x_train["security_delay"] = x_train["security_delay"].astype(int)
x_train["late_aircraft_delay"] = x_train["late_aircraft_delay"].astype(int)
x_train["dep_delay"] = x_train["dep_delay"].astype(int)

x_val["carrier_delay"] = x_val["carrier_delay"].astype(int)
x_val["weather_delay"] = x_val["weather_delay"].astype(int)
x_val["nas_delay"] = x_val["nas_delay"].astype(int)
x_val["security_delay"] = x_val["security_delay"].astype(int)
x_val["late_aircraft_delay"] = x_val["late_aircraft_delay"].astype(int)
x_val["dep_delay"] = x_val["dep_delay"].astype(int)

In [ ]:
x_train.columns
x_train.dtypes

Encode categorical features as a one-hot numeric array

In [ ]:
# define which columns are categorical
categorical_features = ["operating_airline", "origin"]

# define column transformer for encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop = "first"), categorical_features)
    ],
    remainder="passthrough" # Keep the non-categorical features from dataframe
)

# Logistic Regression Pipeline
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000, random_state=123))
])

y_train_array = y_train["cancelled"].to_numpy()

# Fit the model
model.fit(x_train, y_train_array)

# # define one hot encoding
# encoder = OneHotEncoder(sparse=False)
# # transform data
# x_train["operating_airline"] = encoder.fit_transform(x_train["operating_airline"])

In [ ]:
x_train.columns

### Basic Logistic Regression Model - 5 Delay Features

In [ ]:
# Initialize logistic regression model
log_model = LogisticRegression(max_iter=1000, random_state=123, class_weight="balanced")
    # max_iter can be increased if needed

# fit the model to training data
log_model.fit(x_train[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']], y_train)

# predict on validation data
y_pred = log_model.predict(x_val[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']])

# evaluate model
print("Accuracy: ", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))
# print(log_model.coef_)


### SMOTE

In [ ]:
#Apply SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=123)

x_res, y_res = sm.fit_resample(x_train[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']], y_train)

In [ ]:
log_model_res = LogisticRegression(random_state = 123)
log_model_res.fit(x_res, y_res)
y_pred_val_res = log_model_res.predict(x_val[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']])
accuracy_score(y_val,y_pred_val_res)


In [ ]:
from sklearn.metrics import f1_score
print(classification_report(y_val, y_pred_val_res))
print(f"f1 score is {f1_score(y_val, y_pred_val_res, average='binary')}")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
cm = confusion_matrix(y_val, y_pred_val_res)
print(cm)
print("ROC-AUC:", roc_auc_score(y_val, y_pred_val_res))

In [ ]:
# from scipy.stats import chi2_contingency
# contengency_table = pd.crosstab(x_train["operating_airline"], x_train["dot_id_operating_airline"])
# chi2, p_value, dof, expected = chi2_contingency(contengency_table)
# print(f"Chi-squire statistics: {chi2}")
# print(f"P-value: {p_value}")

### SMOTE + Tomek (SMOTEENN)

In [ ]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=123)
X_resampled, y_resampled = smote_enn.fit_resample(x_train[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']], y_train)

In [ ]:
log_model_res2 = LogisticRegression(random_state = 123)
log_model_res2.fit(X_resampled, y_resampled)
y_pred_val_res2 = log_model_res2.predict(x_val[['dep_delay', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'operating_airline_AS', 'operating_airline_B6', 'operating_airline_DL',
       'operating_airline_F9', 'operating_airline_G4', 'operating_airline_HA',
       'operating_airline_NK', 'operating_airline_UA', 'operating_airline_WN',
       'origin_CLT', 'origin_DEN', 'origin_DFW', 'origin_LAS', 'origin_LAX',
       'origin_MCO', 'origin_MIA', 'origin_ORD', 'origin_PHX', 'crs_dep_hour']])
accuracy_score(y_val,y_pred_val_res2)

In [ ]:
from sklearn.metrics import f1_score
print(classification_report(y_val, y_pred_val_res2))
print(f"f1 score is {f1_score(y_val, y_pred_val_res2, average='binary')}")
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
cm2 = confusion_matrix(y_val, y_pred_val_res2)
print(cm2)
print("ROC-AUC:", roc_auc_score(y_val, y_pred_val_res2))

ideas to improve model
- cancelled vs dep_delay (correlation) - justify running 2-way Anova with interactions against dep_delay
- 2-way Anova - interaction between "origin" and "operating_airline" against "dep_delay" to find significant interactions and add back into model 
- interactions
- drop unimformative features (permutation importance or SelectFromModel)
- RandomForest